In [1]:
from quaternion_filters import quaternion
import numpy as np
from sympy import *

In [2]:
def euler2q(roll, pitch, heading):
    q = quaternion.Quaternion.axis_angle(np.array([0, 0, 1.]), heading)
    q = q*quaternion.Quaternion.axis_angle(np.array([0, 1., 0]), pitch)
    q = q*quaternion.Quaternion.axis_angle(np.array([1., 0, 0]), roll)
    return q

def eulers(q):
    """
    returns: roll, pitch, heading
    """
    a, b, c, d = q.a, q.b, q.c, q.d
    phi = atan2(2*(a*b + c*d),
                        1-2*(b**2 + c**2))
    theta = asin(2*(a*c - d*b))
    psi = atan2(2*(a*d+b*c),
                        1-2*(c**2+d**2))
    return np.array([phi, theta, psi])


def normalize_heading_deg(val_deg):
    if val_deg > 360:
        return normalize_heading(val_deg - 360)
    elif val_deg < 0:
        return normalize_heading(val_deg + 360)
    else:
        return val_deg

In [7]:
roll, pitch, heading = symbols(['roll', 'pitch', 'heading'])
wx, wy, wz = symbols(['wx', 'wy', 'wz'])
ax, ay, az = symbols(['ax', 'ay', 'az'])
mx, my, mz = symbols(['mx', 'my', 'mz'])
G = symbols('G')
dt = symbols('dt')

state = [roll, pitch, heading, ax, ay, az, wx, wy, wz]

quaternion.arctan2 = atan2  # Make quaternion use symbolic functions
quaternion.arcsin = asin
quaternion.sqrt = sqrt
quaternion.sin = sin
quaternion.cos = cos


In [21]:
# Which way is ac for roll
def test():
    heading = 335*np.pi/180
    roll = 60*np.pi/180
    pitch = 0*5*np.pi/180
    q = euler2q(roll, pitch, heading)
    print(eulers(q)*180/np.pi)

    a = np.array([0,0,-G]) + np.array([-sin(heading), cos(heading), 0])*G*tan(roll)

    ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]
    print("body a", ab)
#test()

# Do rotations go the right way?
def test():
    heading = 135*np.pi/180
    roll = 0*np.pi/180
    pitch = 0*5*np.pi/180
    q = euler2q(roll, pitch, heading)
    wx = 10*np.pi/180
    wy = 10*np.pi/180
    wz = 10*np.pi/180
    dt = .1
    qdelta = quaternion.Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)
    #print(eulers(q)*180/np.pi)
    qnext = q * qdelta
    
    print("roll pitch heading", eulers(qnext))
    
test()

roll pitch heading [1.00892971901171 0.991322812393154
 -43.9822113375784 + 57.2957795130823*pi]


In [9]:
q = euler2q(roll, pitch, heading)

qdelta = quaternion.Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)  # small angle approximation of rotation
qnext = q*qdelta

a = np.array([0,0,-G]) + np.array([-sin(heading), cos(heading), 0])*G*tan(roll)

ab = (q.inv()*quaternion.Quaternion.from_vec(a)*q).as_ndarray()[1:]

#wpredict = (q.inv()*quaternion.Quaternion.from_vec([0,0,-G/tas*tan(phi)])*q).as_ndarray()[1:]
wpredict = [wx, wy, wz]
statenext = np.concatenate([eulers(qnext), ab, wpredict])
#statenext = np.array([x.subs(q0**2+q1**2+q2**2+q3**2,1) for x in statenext])
for n in range(len(state)):
    print(f"{state[n]}_next = {statenext[n]}")

roll_next = atan2(2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - dt*wz*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + 1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2)) + 2*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/

az_next = (1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) + G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/

In [10]:
F = np.zeros((len(state), len(state)), dtype=object)
for n in range(len(state)):
    for m in range(len(state)):
        tmp = (diff(statenext[n], state[m]))
        F[n, m] = tmp
        if tmp != 0:
            print(f"F[{n}, {m}] = {tmp}")

F[0, 0] = (-2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - dt*wz*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + 1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2)) - 2*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos

F[0, 2] = (-2*(dt*wx*(0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2)) + dt*wy*(-sin(heading/2)*cos(pitch/2)*cos(roll/2)/2 + 0.5*sin(pitch/2)*sin(roll/2)*cos(heading/2)) - dt*wz*(-0.5*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 0.5*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + sin(roll/2)*cos(heading/2)*cos(pitch/2))*(dt*wx*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wy*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wz*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + 1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - 2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 

F[0, 8] = (-2*(dt*wx*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wy*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 - dt*wy*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 - dt*wz*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + 1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2)) - 2*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos

F[1, 6] = -(dt*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 + 1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2)) + dt*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(dt*wx*(1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))/2 + dt*wy*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 - dt*wz*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + 1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2)) - dt*(1.

F[2, 1] = (-2*(-dt*wx*(-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2)) + dt*wy*(-0.5*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 0.5*sin(pitch/2)*sin(roll/2)*cos(heading/2)) + dt*wz*(0.5*sin(heading/2)*sin(roll/2)*cos(pitch/2) - sin(pitch/2)*cos(heading/2)*cos(roll/2)/2) - 1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*(-dt*wx*(1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/2 + dt*wy*(-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/2 + dt*wz*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))/2 + 1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2)) - 2*(dt*wx*(-0.5*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 0.5*sin(roll/2)*cos(heading/2)*cos(pitch/2)) + dt*wy*(0.5*sin(heading/2)*sin(roll/2)*cos(pitch/2) - sin(pitch/2)*cos(heading/2)*cos(roll/2

...

F[4, 1] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos

F[4, 2] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(he

F[5, 0] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) - 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) + G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(head

F[5, 1] = (-0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(-G*(1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(he

F[5, 2] = (0.5*sin(heading/2)*sin(pitch/2)*sin(roll/2) + 0.5*cos(heading/2)*cos(pitch/2)*cos(roll/2))*(G*(1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) - 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))*sin(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))**2 + (1.0*sin(heading/2)*cos(pitch/2)*cos(roll/2) - 1.0*sin(pitch/2)*sin(roll/2)*cos(heading/2))**2) - G*(-1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) - 1.0*sin(pitch/2)*cos(heading/2)*cos(roll/2))*cos(heading)*tan(roll)/((1.0*sin(heading/2)*sin(pitch/2)*sin(roll/2) + cos(heading/2)*cos(pitch/2)*cos(roll/2))**2 + (-1.0*sin(heading/2)*sin(pitch/2)*cos(roll/2) + 1.0*sin(roll/2)*cos(heading/2)*cos(pitch/2))**2 + (1.0*sin(heading/2)*sin(roll/2)*cos(pitch/2) + 1.0*sin(pitch/2)*cos(heading/2)*cos(

## Mag Update


Kalman equation:
$$y = z - h(x)$$
innovation = measurement - (func translating state to measurement)

In [35]:
q = euler2q(roll, pitch, heading)

mag_inertial = (q * quaternion.Quaternion.from_vec(np.array([mx, my, mz])) * q.inv()).as_ndarray()[1:]
mag_inertial[2] = 0
mag_inertial /= sqrt(mag_inertial[0]**2 + mag_inertial[1]**2)


mag_body = (q.inv() * quaternion.Quaternion.from_vec(mag_inertial) * q).as_ndarray()[1:]

# Only portion that we need to determine Jacobian is transfering "north vector"
# to body coordinates

h = (q.inv() * quaternion.Quaternion.from_vec(np.array([1.0, 0, 0])) * q).as_ndarray()[1:]

y = mag_body - h
H = np.zeros((2, len(state)), dtype=object)
sh2, sp2, sr2 = symbols(['sh2', 'sp2', 'sr2'])
ch2, cp2, cr2 = symbols(['ch2', 'cp2', 'cr2'])
for n in range(2):
    for m in range(len(state)):
        tmp = diff(h[n], state[m])
        H[n, m] = tmp
        if tmp != 0:
            tmp = tmp.subs(sin(heading/2), sh2).subs(sin(pitch/2),0).subs(sin(roll/2),sr2).\
               subs(cos(heading/2),ch2).subs(cos(pitch/2),1.0).subs(cos(roll/2),cr2)
            print(f"H[{n}, {m}] = {tmp}")           

H[0, 0] = 0
H[0, 1] = 0
H[0, 2] = -2.0*ch2*cr2**2*sh2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2) - 2.0*ch2*sh2*sr2**2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2)
H[1, 0] = 4.0*ch2*cr2*sh2*sr2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2)
H[1, 1] = 2.0*ch2**2*cr2*sr2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2) - 2.0*cr2*sh2**2*sr2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2)
H[1, 2] = -1.0*ch2**2*cr2**2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2) + 1.0*ch2**2*sr2**2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2) + 1.0*cr2**2*sh2**2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2) - 1.0*sh2**2*sr2**2/(1.0*ch2**2*cr2**2 + 1.0*ch2**2*sr2**2 + 1.0*cr2**2*sh2**2 + 1.0*sh2**2*sr2**2)
